### Example 1

#### The Traveling Salesman Problem

    considering n points V={0,…,n−1} and a distance matrix Dn×n with elements ci,j∈R+, a solution consists in a set of exactly n (origin, destination) pairs indicating the itinerary of your trip, resulting in the following formulation:


\begin{split}\textrm{Minimize: }   &  \\
&  \sum_{i \in I, j \in I} c_{i,j} \ldotp x_{i,j} \\
\textrm{Subject to: }   &  \\
& \sum_{j \in V \setminus \{i\}} x_{i,j} = 1 \,\,\, \forall i \in V  \\
& \sum_{i \in V \setminus \{j\}} x_{i,j} = 1 \,\,\, \forall j \in V \\
& y_{i} -(n+1)\ldotp x_{i,j} \geq y_{j} -n  \,\,\, \forall i \in V\setminus \{0\}, j \in V\setminus \{0,i\}\\
& x_{i,j} \in \{0,1\} \,\,\, \forall i \in V, j \in V \\
& y_i \geq 0 \,\,\, \forall i \in V\end{split}

In [13]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# names of places to visit
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',     #names of the places to visit are informed
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],     #distances are informed in an upper triangular matrix
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model() #creates an empty MIP model

# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
#all binary decision variables for the selection of arcs are created and their references are stored a n×n matrix named x.
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))  


#Differently from the x variables, y variables are not required to be binary or integral, 
#they can be declared just as continuous variables, the default variable type.
# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# optimizing
model.optimize()

# checking if a solution was found
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

route with total distance 547 found: Antwerp -> Bruges -> Ghent -> Grand-Place de Bruxelles -> Waterloo -> Mons -> Namur -> Dinant -> Remouchamps -> Montagne de Bueren -> C-Mine -> Hasselt -> Leuven -> Mechelen -> Antwerp



### Example 2 
#### The 0/1 Knapsack Problem

    consider the solution of the 0/1 knapsack problem: given a set I of items, each one with a weight $w_{i}$ and estimated profit $p_{i}$, one wants to select a subset with maximum profit such that the summation of the weights of the selected items is less or equal to the knapsack capacity c. Considering a set of decision binary variables $x_{i}$ that receive value 1 if the i-th item is selected, or 0 if not, the resulting mathematical programming formulation is:




\begin{split}\textrm{Maximize: }   &  \\
                               &  \sum_{i \in I} p_i \cdot x_i  \\
\textrm{Subject to: } & \\
                               &  \sum_{i \in I} w_i \cdot x_i \leq c  \\
                               &  x_i \in \{0,1\} \,\,\, \forall i \in I\end{split}


In [10]:
# install mip library using python 
!pip install mip

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [12]:


from mip import Model, xsum, maximize, BINARY

p = [10, 13, 18, 31, 7, 15] # You can create a large list using 'input'
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model("knapsack") # calling the model

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))



selected items: [0, 3]


### Example 3

#### We will be trying to solve the following ILP problem:

##### Min $x_{0}+x_{1}+x_{2}+x_{3}+x_{4}+x_{5}$

##### GIven the following constraints:

$x_{0}+x_{1}\geq1 $

$x_{0}+x_{1}+x_{5}\geq1 $

$x_{2}+x_{3}\geq1 $

$x_{2}+x_{3}+x_{4}\geq1 $

$x_{3}+x_{4}+x_{5}\geq1 $

$x_{1}+x_{4}+x_{5}\geq1 $

$x_{0},x_{1},x_{2},x_{3},x_{4},x_{5}\in Z$

##### Now, GLPK ILP solver assumes the following form of the problem.

In [ ]:
# importing the libraries
from cvxopt.glpk import ilp
import numpy as np
from cvxopt import matrix


In [11]:

c=matrix(np.ones(6,dtype=float))
print (c)

[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]



In [12]:

coeff=np.array([[1,1,0,0,0,0],
                [1,1,0,0,0,1],
                [0,0,1,1,0,0],
                [0,0,1,1,1,0],
                [0,0,0,1,1,1],
                [0,1,0,0,1,1]
                ],dtype=float)

In [14]:

G=matrix(-coeff)
print (G)

[-1.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-1.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00]
[-0.00e+00 -0.00e+00 -1.00e+00 -1.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -1.00e+00 -1.00e+00 -1.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -1.00e+00 -1.00e+00]
[-0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -1.00e+00]



In [15]:

h=matrix(-1*np.ones(6))

In [18]:

I=set(range(6))

In [19]:
B=set()
print (I,B)

{0, 1, 2, 3, 4, 5} set()


In [20]:

(status,x)=ilp(c,G,h,matrix(1., (0,6)),matrix(1., (0,1)),I,B)

In [21]:

status

'optimal'

In [22]:

print (x)

[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]



In [23]:
(status,x)=ilp(c,G,h,matrix(1., (0,6)),matrix(1., (0,1)),B,I)

In [25]:
print (status)
print (x)

optimal
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]

